## **3. Pondération statistique** (TF-IDF / OKapiBM25)  

https://stackoverflow.com/questions/46580932/calculate-tf-idf-using-sklearn-for-n-grams-in-python  
http://scikit-learn.sourceforge.net/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn-feature-extraction-text-tfidfvectorizer  
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

https://pypi.org/project/rank-bm25/

In [17]:
path = '../04-filtrage/output/'
acteur = 'inesss'
tag = ''

if tag:
    csv_file = acteur + '_' + tag + '_significant-collocations.csv'

else:
    csv_file = acteur + '_significant-collocations.csv'

### **Lire le vocabulaire** (termes retenus au prétraitement)

In [18]:
from pandas import *

with open(path+csv_file, encoding='utf-8') as f:
    csv = read_csv(f)[["Collocation", "Structure syntaxique", "Fréquence"]] # "LLR", "p-value"]]

csv

,Collocation,Structure syntaxique,Fréquence
0,réponses rapides,NOM ADJ,1319
1,cours d'évaluation,NOM PRP NOM,1288
2,réponses rapides disponibles,NOM ADJ ADJ,1179
3,usage optimal,NOM ADJ,1138
4,bicarbonate de sodium,NOM PRP NOM,980
...,...,...,...
6665,année de vie,NOM PRP NOM,6
6666,entrée en vigueur,NOM PRP NOM,6
6667,fumarate de nom,NOM PRP NOM,6
6668,utilisation et résultats cliniques en contexte...,NOM KON NOM ADJ PRP NOM ADJ,6


In [19]:
vocabulaire = [t.lower() for t in list(csv['Collocation'])]

In [20]:
print('On a un vocabulaire de {} formes.'.format(len(vocabulaire)))
vocabulaire

On a un vocabulaire de 6670 formes.


['réponses rapides',
 "cours d'évaluation",
 'réponses rapides disponibles',
 'usage optimal',
 'bicarbonate de sodium',
 'jamp lurasidone',
 "attente d'évaluation",
 'listes des médicaments',
 'version française',
 'services sociaux',
 "demande en cours d'évaluation",
 'demande en cours',
 "guide d'usage",
 "guide d'usage optimal",
 'demande en attente',
 'inesss express',
 'pré ac en attente',
 'dénomination commune',
 'nom du fabricant',
 "recommandation de l'inesss",
 'besoins hospitaliers',
 'décision du ministre',
 'sulfonate calcique de jamp',
 'citrate liquid',
 'calcium polystyrene',
 'sodium pour injection',
 'calcium polystyrene sulfonate',
 'bicarbonate de sodium pour injection',
 'injection de glycopyrrolate',
 'apo eletriptan',
 'sulfonate calcique',
 'dépôt pour évaluation',
 "attente d'un dépôt",
 "attente d'un dépôt pour évaluation",
 'adultes atteints',
 'citrate liquid angita calcium',
 'apo eletriptan apotex',
 'fumarate de apo eletriptan apotex',
 'jamp glycopyrrol

### **Lire le corpus**

In [21]:
import os, shutil, re
from pathlib import Path
from os import path
from pandas import *

base_path = '../03-corpus/2-data/1-fr/'
if tag:
    base_path = path.join(base_path, acteur, acteur + '_' + tag + '.csv')

else:
    base_path = path.join(base_path, acteur +  '.csv')
        
with open(base_path, "r", encoding = "UTF-8") as f:
    data = read_csv(base_path, sep=',')
    text = data['text'].tolist()

In [22]:
text = text[:round(len(text))]

nb_docs = len(text)

print("On a donc un corpus de {} documents.".format(nb_docs))

On a donc un corpus de 1226 documents.


### **Nettoyage**

In [23]:
corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in text]
    
punct = '[!#$%&\(\)•►*+,-\/:;<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

corpus = [re.sub(punct, ' ', t).replace("' ", "'" ).replace("'", "'") for t in corpus]
corpus = [re.sub(spaces, ' ', t) for t in corpus]
corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in corpus]
corpus = [re.sub(spaces, ' ', t) for t in corpus]
corpus = [re.sub(phones, ' ', t) for t in corpus]
corpus = [re.sub(postals, ' ', t) for t in corpus]
corpus = [re.sub(punct, ' ', t) for t in corpus]
corpus = [t.replace("  ", " " ) for t in corpus]

### **Appliquer le prétraitement**
Si les termes passées comme vocabulaire sont lemmatisés, changer le paramètre lem pour True au moment d'appliquer la fonction nlp(corpus)  
Le TfIdfVectorizer de sklearn va extraire lui-mêmes les ngrammes, faire le filtrage des mots fonctionnels et calculer le tf-idf pour nos termes d'intérêt ;  
Or, si les termes qu'on lui donne comme vocabulaire ont été lemmatisés, on veut donc aussi lui passer un corpus lemmatisé.

In [24]:
import nltk
from nltk.tokenize import RegexpTokenizer
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

def nlp(corpus, lem=False): 
    if not lem:
        # Tokenisation
        tokenizer = RegexpTokenizer(r"\w\'|\w+")

        tokens = [tokenizer.tokenize(doc) for  doc in corpus]
        len_corpus = len(nltk.flatten(tokens))
        print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len_corpus))

        return tokens

    else:
        # POS tagging
        input = [" ".join(nltk.flatten(doc)).replace("' ", "'") for doc in tokens]
        import treetaggerwrapper
        tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')

        path = '../04-filtrage/mapping_treeTagger_lefff.csv'

        with open(path) as f:
            csv = read_csv(f)

        treeTag = [term for term in csv['TreeTagger'].tolist()] 
        lefff = [term for term in csv['Lefff'].tolist()]

        mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

        tagged= [tagger.tag_text(doc) for doc in input]

        tuples_doc = []
        for doc in tagged:
            tuples = []
            for t in doc:
                token = t.split('\t')[0]
                pos = mapping[t.split('\t')[1]]

                tuples.append([token, pos])
            tuples_doc.append(tuples)

        #Lemmatisation
        lemmatizer = FrenchLefffLemmatizer()
        docs_lemmas = []

        for doc in tuples_doc:
            doc_lemma = []
            for t in doc:
                term_lemmatized = ""
                if(lemmatizer.lemmatize(t[0], t[1]) == []):
                    term_lemmatized = lemmatizer.lemmatize(t[0])
                else:
                    term_lemmatized = lemmatizer.lemmatize(t[0], t[1])[0][0] # [0][0] pour avoir le lemme seul et non (lemme, pos)
            
                if len(term_lemmatized) >1 :
                    doc_lemma.append(term_lemmatized)
            docs_lemmas.append(doc_lemma)

        docs_lemmas = [" ".join(doc) for doc in docs_lemmas]

        return docs_lemmas

In [25]:
# corpus = nlp(corpus)

In [26]:
file_path = '../04-filtrage/mwe_stopwords.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mwe_sw = [t.lower().strip('\n') for t in f.readlines()]

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

# max_df : ignore words that appear in 85% of documents, 
# min df:  ignore words that appear in less than 1% of documents 
# vocabulary = vocabulaire

# Sans utiliser le vocabulaire
# tfidf = TfidfVectorizer(min_df=0.1, stop_words=None, ngram_range=(2,4), max_df=0.85, use_idf=True)

# def identity_tokenizer(text):
#     return text
# tokenizer=identity_tokenizer

# vocabulary = vocabulaire
tfidf = TfidfVectorizer(vocabulary = set(vocabulaire),ngram_range=(1,12), use_idf=True, lowercase=False)
tfs = tfidf.fit_transform(corpus)

In [28]:
features_names = tfidf.get_feature_names_out()
corpus_index = [corpus.index(n) for n in corpus]

import pandas as pd
df = pd.DataFrame(tfs.T.todense(), index=features_names, columns=corpus_index).transpose()

In [29]:
df

,aberration génomique,aberration génomique tumorale,aberration génomique tumorale egfr,absence de données,absence de détérioration,abus physique,acceptabilité professionnelle,acceptabilité professionnelle et sociale,acceptabilité professionnelle et sociale des productions,accident vasculaire,...,évolution du risque,évolution du risque d'hospitalisation,évolution fonctionnelle,évolution la médecine,évolution rapide,îlots de langerhans,îlots de langerhans chez les personnes atteintes,œdème maculaire,œdème maculaire diabétique,œdème maculaire diabétique pré
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# from pathlib import Path

# base_path = '../05-transformation/' + acteur + '/'
# Path(base_path).mkdir(parents=True, exist_ok=True)

# if sous_corpus:
#     path = base_path + tag + '/'
#     titre = tag

# else:
#     titre = acteur

# df.to_csv(base_path + titre + '_matrice-TFIDF.csv')

In [31]:
terms_weighted = []
rows, cols = tfs.nonzero()
for row, col in zip(rows,cols):
    terms_weighted.append([features_names[col], tfs[row,col]])

terms_weighted = DataFrame(terms_weighted, columns=['Collocation', 'TF-IDF'])
terms_weighted.sort_values(["TF-IDF"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

In [32]:
terms_weighted = terms_weighted.drop_duplicates(keep='first')

terms_weighted

,Collocation,TF-IDF
5776,usage optimal,1.000000
6639,développement durable,1.000000
8278,ministre sur les génériques,0.987677
78348,formules nutritives,0.928014
7989,professionnelle scientifique,0.923344
...,...,...
19105,patients atteints,0.001125
18947,réponse rapide covid,0.001088
19534,affections post covid,0.001083
19535,affections post,0.001083


In [33]:
terms_weighted = pd.merge(csv, terms_weighted, on='Collocation').drop_duplicates(
  subset = ['Collocation', 'Fréquence'],
  keep = 'first').reset_index(drop = True)

In [34]:
terms_weighted

,Collocation,Structure syntaxique,Fréquence,TF-IDF
0,réponses rapides,NOM ADJ,1319,0.649766
1,réponses rapides disponibles,NOM ADJ ADJ,1179,0.550419
2,usage optimal,NOM ADJ,1138,1.000000
3,bicarbonate de sodium,NOM PRP NOM,980,0.263692
4,jamp lurasidone,NOM ADJ,963,0.263692
...,...,...,...,...
5251,année de vie,NOM PRP NOM,6,0.266166
5252,entrée en vigueur,NOM PRP NOM,6,0.065645
5253,fumarate de nom,NOM PRP NOM,6,0.298408
5254,utilisation et résultats cliniques en contexte...,NOM KON NOM ADJ PRP NOM ADJ,6,0.192468


**Test : clustering (documents)**

In [35]:
def cluster_text(corpus):
    # vocabulary = vocabulaire
    vectorizer = TfidfVectorizer(vocabulary = set(vocabulaire), tokenizer=identity_tokenizer, ngram_range=(2,5), use_idf=True, lowercase=False, stop_words= mwe_sw)
    X = vectorizer.fit_transform(corpus)

    import matplotlib.pyplot as plt
    from sklearn.cluster import KMeans
    Sum_of_squared_distances = []
    K = range(2,10)
    for k in K:
       km = KMeans(n_clusters=k, max_iter=200, n_init=10)
       km = km.fit(X)
       Sum_of_squared_distances.append(km.inertia_)
    plt.plot(K, Sum_of_squared_distances, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Sum_of_squared_distances')
    plt.title('Elbow Method For Optimal k')
    plt.show()

    print('How many clusters do you want to use?')
    true_k = int(input())
    model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=10)
    model.fit(X)

    labels=model.labels_
    clusters=pd.DataFrame(list(zip(text,labels)),columns=['title','cluster'])
    #print(clusters.sort_values(by=['cluster']))

    for i in range(true_k):
        print(clusters[clusters['cluster'] == i])
        clusters.to_csv('../06-clustering/' + acteur + '_clusters.csv')
        
    return

In [36]:
#cluster_text(corpus)

## **OKapi BM25**
https://hal.archives-ouvertes.fr/hal-00760158 

In [37]:
from rank_bm25 import BM25Okapi

In [38]:
corpus = nlp(corpus)
bm25 = BM25Okapi(corpus)

Avec le RegExpTokenizer, notre corpus contient 641308 tokens.


In [39]:
#tokenizer = RegexpTokenizer(r"\w\'|\w+")
tokenized_queries = [t.split() for t in set(vocabulaire)]

features_names = [t for t in set(vocabulaire)]
corpus_index = [corpus.index(n) for n in corpus]

tab = [bm25.get_scores(query) for query in tokenized_queries]
df = pd.DataFrame(tab, index=features_names, columns=corpus_index).transpose()

In [40]:
tokenized_queries

[['diagnostic', 'confirmé'],
 ['trauma', 'patients'],
 ['régime', 'public'],
 ['solution', "d'intelligence", 'artificielle'],
 ['cancer', 'infectées', 'par', 'le', 'sras', 'cov'],
 ['dispositifs', "d'accès", 'vasculaire'],
 ['conseil', 'en', 'traumatologie'],
 ['rétention', 'urinaire'],
 ['herpès', 'génital'],
 ['cardiologie', 'et', 'maladies', 'neurovasculaires'],
 ['publication', 'de', "l'état", 'des', 'pratiques'],
 ['évaluation', 'foquest', 'méthylphénidate', 'chlorhydrate'],
 ['formule', 'nutritive'],
 ['traitement', 'de', "l'oedème"],
 ['blessures', 'médullaires'],
 ['validation', 'analytique', 'de', 'tests'],
 ['instituts', 'de', 'recherche', 'en', 'santé'],
 ['réactif', 'quantitatif', 'du', 'temps'],
 ['cardio', 'ou', 'neuro', 'vasculaires'],
 ['nom', 'du', 'fabricant', 'novartis'],
 ['présence', 'de', 'douleurs'],
 ['rinçage', 'pour', 'les', 'dispositifs', "d'accès"],
 ['prise', 'en', 'charge', 'de', "l'infarctus"],
 ['comité', "d'experts"],
 ['réanimation', 'cardiorespiratoir

In [41]:
df

,diagnostic confirmé,trauma patients,régime public,solution d'intelligence artificielle,cancer infectées par le sras cov,dispositifs d'accès vasculaire,conseil en traumatologie,rétention urinaire,herpès génital,cardiologie et maladies neurovasculaires,...,sonde urinaire chez un adulte,incyte en association avec la lénalidomide,comportement meilleures,critical care medicine,analyses de laboratoire au répertoire québécois,arrondissement des doses d'agents,fabricant novartis,alcool troubles neurocognitifs document complémentaire,durée de vie,évaluation ultomiris
0,0.000000,1.232356,0.000000,0.0,8.109969,0.000000,6.543854,0.000000,0.0,6.289166,...,2.511007,2.511007,0.0,0.0,3.425369,3.067883,0.000000,0.000000,3.425369,1.302201
0,0.000000,1.232356,0.000000,0.0,8.109969,0.000000,6.543854,0.000000,0.0,6.289166,...,2.511007,2.511007,0.0,0.0,3.425369,3.067883,0.000000,0.000000,3.425369,1.302201
0,0.000000,1.232356,0.000000,0.0,8.109969,0.000000,6.543854,0.000000,0.0,6.289166,...,2.511007,2.511007,0.0,0.0,3.425369,3.067883,0.000000,0.000000,3.425369,1.302201
3,1.585282,3.776687,0.000000,0.0,3.535006,2.077158,8.756992,4.038318,0.0,11.545511,...,6.612289,6.160739,0.0,0.0,10.352865,3.327449,0.000000,9.392838,3.390084,1.596599
4,0.000000,0.000000,0.000000,0.0,5.413158,0.000000,2.706826,0.000000,0.0,3.005098,...,1.809041,5.838985,0.0,0.0,6.263093,3.154049,0.000000,0.000000,3.377245,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,0.000000,0.000000,2.807276,0.0,2.545371,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,2.545371,0.0,0.0,5.869992,1.968600,0.408552,0.000000,3.324622,0.000000
686,0.000000,0.000000,0.000000,0.0,2.570779,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,2.999615,0.0,0.0,5.945713,0.000000,0.414895,0.000000,3.374934,0.000000
1223,0.000000,0.000000,0.000000,0.0,2.568807,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,2.998272,0.0,0.0,5.943174,0.000000,0.414401,0.000000,3.374367,0.000000
857,0.000000,0.000000,0.000000,0.0,2.120735,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,6.034552,2.669159,0.440126,0.000000,3.365394,0.000000


In [42]:
#df.to_csv(base_path + titre + '_matrice-OkapiBM25.csv') # Si on veut avoir la matrice (mais le fichier peut être très volumineux)

In [43]:
terms_okapi = {term: df[term].max() for term in df}

In [44]:
terms_okapi

{'diagnostic confirmé': 10.953869653208749,
 'trauma patients': 9.510870872554289,
 'régime public': 9.461682222679254,
 "solution d'intelligence artificielle": 14.13648897755764,
 'cancer infectées par le sras cov': 34.56908602070112,
 "dispositifs d'accès vasculaire": 10.54573923365426,
 'conseil en traumatologie': 13.127230284192766,
 'rétention urinaire': 12.466973350047947,
 'herpès génital': 13.83570112169787,
 'cardiologie et maladies neurovasculaires': 23.993398941487513,
 "publication de l'état des pratiques": 14.082523130654216,
 'évaluation foquest méthylphénidate chlorhydrate': 5.709888502219647,
 'formule nutritive': 13.636504985388681,
 "traitement de l'oedème": 6.7777936973800355,
 'blessures médullaires': 12.401140916174867,
 'validation analytique de tests': 20.808310752496595,
 'instituts de recherche en santé': 19.88202668384567,
 'réactif quantitatif du temps': 16.030637546965117,
 'cardio ou neuro vasculaires': 22.937198541908227,
 'nom du fabricant novartis': 8.67

In [45]:
tab = DataFrame(terms_okapi.items(), columns=['Collocation', 'OkapiBM25'])
tab

,Collocation,OkapiBM25
0,diagnostic confirmé,10.953870
1,trauma patients,9.510871
2,régime public,9.461682
3,solution d'intelligence artificielle,14.136489
4,cancer infectées par le sras cov,34.569086
...,...,...
6665,arrondissement des doses d'agents,16.065151
6666,fabricant novartis,5.588819
6667,alcool troubles neurocognitifs document complé...,20.569432
6668,durée de vie,11.555622


In [46]:
tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

tab = pd.merge(terms_weighted, tab, on="Collocation")
tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

tab

,Collocation,Structure syntaxique,Fréquence,TF-IDF,OkapiBM25
4255,seringues bd saline sp nacl,NOM ADJ ADJ NOM ADJ,7,0.590901,64.530807
3937,vertu de la loi sur la protection de la jeunes...,NOM PRP DET:ART NOM PRP DET:ART NOM PRP DET:AR...,8,0.159432,55.850221
4831,système de mesure du glucose en continu guardi...,NOM PRP NOM PRP:det NOM PRP NOM ADJ ADJ,6,0.156902,53.401714
2331,services en vertu de la loi sur la protection ...,NOM PRP NOM PRP DET:ART NOM PRP DET:ART NOM PR...,17,0.203912,50.589724
4750,pertinence des envois de prélèvements en patho...,NOM PRP:det NOM PRP NOM PRP NOM PRP NOM,6,0.272956,50.390892
...,...,...,...,...,...
57,apotex élétriptan,NOM ADJ,474,0.133841,1.236537
23,apo eletriptan,NOM ADJ,489,0.131846,1.220759
12,dénomination commune,NOM ADJ,564,0.129499,1.203497
38,évaluations récemment,NOM ADV,474,0.133841,1.158830


In [47]:
base_path = '../05-transformation/'

In [48]:
if tag:
    file_path = base_path + acteur + '_' + tag + '_weighting_OKapiBM25.csv'

else: 
    file_path = base_path + acteur  + '_weighting_OKapiBM25.csv'
tab.to_csv(file_path)

In [49]:
tab

,Collocation,Structure syntaxique,Fréquence,TF-IDF,OkapiBM25
4255,seringues bd saline sp nacl,NOM ADJ ADJ NOM ADJ,7,0.590901,64.530807
3937,vertu de la loi sur la protection de la jeunes...,NOM PRP DET:ART NOM PRP DET:ART NOM PRP DET:AR...,8,0.159432,55.850221
4831,système de mesure du glucose en continu guardi...,NOM PRP NOM PRP:det NOM PRP NOM ADJ ADJ,6,0.156902,53.401714
2331,services en vertu de la loi sur la protection ...,NOM PRP NOM PRP DET:ART NOM PRP DET:ART NOM PR...,17,0.203912,50.589724
4750,pertinence des envois de prélèvements en patho...,NOM PRP:det NOM PRP NOM PRP NOM PRP NOM,6,0.272956,50.390892
...,...,...,...,...,...
57,apotex élétriptan,NOM ADJ,474,0.133841,1.236537
23,apo eletriptan,NOM ADJ,489,0.131846,1.220759
12,dénomination commune,NOM ADJ,564,0.129499,1.203497
38,évaluations récemment,NOM ADV,474,0.133841,1.158830
